In [1]:
from jaxspec.ispec_synthetic_grid import *

In [8]:
#ranges = define_grid_ranges(tstep=2000, gstep=2, fstep=1.)
ranges = define_grid_ranges()

In [9]:
#precompute_synthetic_grid(ranges, 1400, 1800, 120000, "ispecgrid_hband", "turbospectrum", number_of_processes=10)
#precompute_synthetic_grid(ranges, 1400, 1800, 120000, "ispecgrid_hband", "synthe", atmosphere_model='atlas', number_of_processes=10)

In [10]:
precompute_synthetic_grid(ranges, 500, 600, 80000, "ispecgrid_gaoes", "turbospectrum", number_of_processes=10)

# step wave (AA): 0.012500156252031047
# resolution: 439994.49999725807
# atmosphere model: marcs
